# Loading the Movies Dataset (With the keywords column)

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/CS247 Project/ml-100k"


Mounted at /content/drive


In [ ]:
 movies_df = pd.read_csv("/content/drive/MyDrive/CS247 Project/1M_movies_with_plots.csv")

In [ ]:
movies_df

,Unnamed: 0.1,Unnamed: 0,movie_id,title,genres,movie_plot,processed_plot,keywords
0,0,0,1,Toy Story (1995),Animation Children's Comedy,"Sentient when humans are not around, a group o...",sentient human around group toy prepar move ne...,Animation Children's Comedy sentient human aro...
1,1,1,2,Jumanji (1995),Adventure Children's Fantasy,"In 1969, Alan Parrish lives in Brantford, New ...",alan parrish live brantford new hampshir paren...,Adventure Children's Fantasy alan parrish live...
2,2,2,3,Grumpier Old Men (1995),Comedy Romance,The feud between Max and John has cooled and t...,feud max john cool becom good friend children ...,Comedy Romance feud max john cool becom good f...
3,3,3,4,Waiting to Exhale (1995),Comedy Drama,"Four friends (Savannah, Robin, Bernadine, and ...",four friend savannah robin bernadin gloria get...,Comedy Drama four friend savannah robin bernad...
4,4,4,5,Father of the Bride Part II (1995),Comedy,"Four years after the events of the first film,...",four year event first film anni bank tell fami...,Comedy four year event first film anni bank te...
...,...,...,...,...,...,...,...,...
3152,3152,3878,3948,Meet the Parents (2000),Comedy,"Greg Focker, a Jewish American nurse living in...",greg focker jewish american nurs live chicago ...,Comedy greg focker jewish american nurs live c...
3153,3153,3879,3949,Requiem for a Dream (2000),Drama,"Sara Goldfarb, a widow who lives alone in a Br...",sara goldfarb widow live alon brighton beach a...,Drama sara goldfarb widow live alon brighton b...
3154,3154,3880,3950,Tigerland (2000),Drama,"In September 1971, the United States is losing...",septemb unit state lose vietnam war roland boz...,Drama septemb unit state lose vietnam war rola...
3155,3155,3881,3951,Two Family House (2000),Drama,An unseen narrator looks back to the year 1956...,unseen narrat look back year elm park neighbor...,Drama unseen narrat look back year elm park ne...


In [ ]:
 movies_df = pd.read_csv("/content/drive/MyDrive/CS247 Project/1M_movies_with_plots.csv")

In [ ]:
movies_df.shape

(3157, 8)

# Generating BERT Embeddings


In [ ]:
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
random_seed = 42
random.seed(random_seed)

torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
class generateBERTEmbeddings():

  # The function returns a dictionary containing the token IDs and attention masks
  def encodeData(self, inputData):

    encoding = tokenizer.batch_encode_plus(
            inputData,
            padding=True,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
    )

    return encoding

  # Returns a tesor that contains embeddings for each word
  def generateWordEmbeddings(self, input_ids, attention_mask):
    with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask)
      word_embeddings = outputs.last_hidden_state  # This contains the embeddings

    return word_embeddings

  # Generate a sentence embedding by averaging out all the word embeddings.
  def getSentenceEmbedding(self, word_embeddings):
    sentence_embedding = word_embeddings.mean(dim=1)

    return sentence_embedding

In [ ]:
# Get the movie keywords column
data = movies_df['keywords']
data = list(data)
print(f"{type(data)}, {len(data)}, {data[0]}")

<class 'list'>, 3157, Animation Children's Comedy sentient human around group toy prepar move new hous owner child andi davi younger sister molli singl mother davi sheriff woodi andi favorit toy de facto leader send sarg green armi men spi andi birthday parti babi monitor toy nervou receiv new toy could replac toy includ potato head slinki dog rex tyrannosaur hamm piggi bank bo peep porcelain doll andi receiv buzz lightyear action figur believ actual space ranger know toy buzz impress other featur becom andi new favorit toy make woodi day move andi famili plan dinner pizza planet ensur andi bring along buzz woodi tri use car rc knock buzz behind desk accident knock bedroom window instead toy except bo slinki believ woodi deliber tri kill buzz andi take woodi furiou buzz confront car two fight fall car left quarrel hitch ride restaur pizza planet deliveri truck buzz mistakenli believ claw crane full littl green men true rocket andi sadist neighbor sid destroy toy fun captur two crane ta

In [ ]:
bertEmbedding = generateBERTEmbeddings()

encoding = bertEmbedding.encodeData(data)

# Contains encoded tokens of each data point
input_ids = encoding['input_ids'].to(device)  # Token IDs

# Contains attention masks of each data point
attention_mask = encoding['attention_mask'].to(device)
print(len(input_ids))
print(len(attention_mask))


3157
3157


In [ ]:
print(f"{len(attention_mask[:10])}, {len(input_ids[0])}")

10, 512


In [ ]:

num_samples = len(input_ids)
batch_size = 100

with open('1M_Movies_BERT_movie_embeddings.csv', 'a') as f:
    # Iterate over the input tensors and split them into batches
    for i in range(0, num_samples, batch_size):

        input_ids_batch = input_ids[i:i+batch_size]
        attention_mask_batch = attention_mask[i:i+batch_size]

        print(f"{len(input_ids_batch)}, {len(attention_mask_batch)}")


        movie_embeddings_batch = bertEmbedding.generateWordEmbeddings(input_ids_batch, attention_mask_batch)


        sentence_embedding_batch = movie_embeddings_batch.mean(dim=1)
        sentence_embedding_batch = sentence_embedding_batch.to('cpu')
        df_batch = pd.DataFrame(sentence_embedding_batch.numpy())
        df_batch.to_csv(f, header=False, index=False)

        print(f"Batch {i//batch_size} processed and appended to movie_embeddings.csv")


100, 100
Batch 0 processed and appended to movie_embeddings.csv
100, 100
Batch 1 processed and appended to movie_embeddings.csv
100, 100
Batch 2 processed and appended to movie_embeddings.csv
100, 100
Batch 3 processed and appended to movie_embeddings.csv
100, 100
Batch 4 processed and appended to movie_embeddings.csv
100, 100
Batch 5 processed and appended to movie_embeddings.csv
100, 100
Batch 6 processed and appended to movie_embeddings.csv
100, 100
Batch 7 processed and appended to movie_embeddings.csv
100, 100
Batch 8 processed and appended to movie_embeddings.csv
100, 100
Batch 9 processed and appended to movie_embeddings.csv
100, 100
Batch 10 processed and appended to movie_embeddings.csv
100, 100
Batch 11 processed and appended to movie_embeddings.csv
100, 100
Batch 12 processed and appended to movie_embeddings.csv
100, 100
Batch 13 processed and appended to movie_embeddings.csv
100, 100
Batch 14 processed and appended to movie_embeddings.csv
100, 100
Batch 15 processed and app

In [ ]:
embeddings = pd.read_csv("/content/1M_Movies_BERT_movie_embeddings.csv")

In [ ]:
display(embeddings)

,-0.35085583,0.02842405,0.5428292,0.030510647,0.26108494,-0.022955192,0.040764835,-0.058894627,-0.09465468,-0.2400521,...,-0.3837117,-0.33460864,0.10507421,0.15178531,0.022968955,0.006760592,-0.6391679,-0.0027313083,0.13903043,0.0017767262
0,-0.384977,-0.085811,0.500225,-0.151379,0.409498,-0.056123,0.203694,0.136267,-0.058841,-0.360539,...,-0.302339,-0.324584,0.040495,0.278048,-0.141157,-0.118833,-0.605397,-0.011370,0.011629,-0.029570
1,-0.142432,-0.020944,0.562685,-0.071085,0.280003,-0.010711,0.229678,0.087884,-0.047246,-0.376674,...,-0.267885,-0.313577,-0.046802,0.153700,-0.081470,-0.057364,-0.406755,-0.123424,-0.015184,-0.128891
2,-0.120158,-0.100989,0.499671,0.061887,0.335525,0.014330,0.163607,0.031799,0.043397,-0.314905,...,-0.101084,-0.331897,-0.053286,-0.052453,-0.024122,-0.096377,-0.389682,-0.070612,-0.028767,-0.120591
3,-0.441387,0.013410,0.524977,-0.173162,0.332954,-0.050337,0.182995,0.170893,0.097232,-0.426208,...,-0.414873,-0.437128,0.112966,0.239903,-0.172287,-0.235706,-0.804357,-0.241305,0.079578,0.022656
4,-0.358970,-0.077872,0.459222,-0.090447,0.399892,-0.096611,0.215167,-0.007046,0.177999,-0.261209,...,-0.398012,-0.356410,0.164383,0.069456,-0.132906,-0.093093,-0.588570,-0.105745,-0.041850,-0.236947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,-0.302594,0.219595,0.457388,-0.204967,0.468501,-0.109716,0.238030,0.118564,0.093600,-0.282418,...,-0.419852,-0.416473,0.049059,0.219200,-0.087787,-0.155575,-0.625097,-0.099385,0.157751,-0.064225
3152,-0.283363,0.037498,0.500044,-0.099773,0.436480,0.017976,0.193944,0.088570,-0.147175,-0.334310,...,-0.427161,-0.471797,0.019724,0.203145,-0.070125,-0.150008,-0.664523,-0.105678,-0.059383,-0.132790
3153,-0.328665,-0.167144,0.443649,-0.163901,0.252768,-0.062206,0.206540,-0.057985,-0.001148,-0.333205,...,-0.381765,-0.300256,-0.079207,0.024022,-0.117044,-0.129184,-0.702894,0.108403,-0.169871,-0.115652
3154,-0.190956,-0.074954,0.439444,-0.066282,0.244499,0.052210,0.166070,0.014230,0.126610,-0.259632,...,-0.035885,-0.377578,0.061973,-0.040141,-0.079614,0.012161,-0.383447,-0.123110,-0.118170,0.012001


In [ ]:
len(embeddings)

3156